In [1]:
from datetime import datetime
import pandas as pd
from pandas import Series
import numpy as np
import matplotlib.pyplot as plt
from iexfinance import get_historical_data
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, roc_auc_score, f1_score, confusion_matrix, r2_score, mean_squared_error
from sklearn.svm import SVC
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
import itertools
from ipywidgets import interact
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
init_notebook_mode(connected=True)
np.random.seed(7)

In [2]:
def get_iex_data(stock_list, start=datetime(2013,9,1), end=datetime(2018,10,5)):
    return_list = []
    for i in stock_list:
        df = pd.DataFrame(get_historical_data(i, start, end, output_format='pandas')).interpolate()
        df['ticker'] = i
        return_list.append(df)
    return return_list
etf_list = ['SPY','IVV','VTI','VOO','QQQ','VEA','EFA','IEFA','VWO','AGG','IJH','IEMG','IWM','IJR','VTV','IWF','IWD','VUG','BND','LQD']

In [5]:
data = clean_data(get_iex_data(etf_list))
data = add_lags(data, 5)
data = add_indicators(data)

In [4]:
def lstm_clean_data(data):
    for i in range(len(data)):
        data[i] = data[i].reset_index().dropna()
        data[i]['date'] = pd.to_datetime(data[i]['date'])
#         data[i]['friday'] = ((((data[i]['date'].diff(-1).astype('int'))/86400000000000)+1)/(-2)).astype('int')
        data[i]['Reg_Target'] = data[i]['close'].shift(-1)
#         data[i] = pd.merge(data[i], prices, on='date')
    return data
def clean_data(data):
    for i in range(len(data)):
        data[i] = data[i].reset_index().dropna()
        data[i]['date'] = pd.to_datetime(data[i]['date'])
        data[i] = data[i].set_index('date')
        data[i]['Clf_Target'] = (np.sign(-data[i]['close'].diff(periods=-1))+1)/2
    return data

def add_lags(etf_list, window):
    for i in range(len(etf_list)):
        for n in range(1,window+1):
            etf_list[i]['{}day_change'.format(n)] = -etf_list[i]['close'].diff(periods=n)
    return etf_list

def RSI(series, period):
    delta = series.diff().dropna()
    u = delta * 0
    d = u.copy()
    u[delta > 0] = delta[delta > 0]
    d[delta < 0] = -delta[delta < 0]
    u[u.index[period-1]] = np.mean( u[:period] )
    u = u.drop(u.index[:(period-1)])
    d[d.index[period-1]] = np.mean( d[:period] )
    d = d.drop(d.index[:(period-1)])
    rs = Series.ewm(u, com=period-1, adjust=False).mean() / \
    Series.ewm(d, com=period-1, adjust=False).mean()
    return 100 - 100 / (1 + rs)

def add_indicators(etf_list):
    for i in range(len(etf_list)):
        etf_list[i]['ewma7'] = etf_list[i]['close'].ewm(span=7,min_periods=0,adjust=True,ignore_na=False).mean()
        etf_list[i]['ewma50'] = etf_list[i]['close'].ewm(span=50,min_periods=0,adjust=True,ignore_na=False).mean()
        etf_list[i]['ewma200'] = etf_list[i]['close'].ewm(span=200,min_periods=0,adjust=True,ignore_na=False).mean()
        etf_list[i]['MACD'] = (etf_list[i]['close'].ewm(span=12,min_periods=0,adjust=True,ignore_na=False).mean() - \
                               etf_list[i]['close'].ewm(span=26,min_periods=0,adjust=True,ignore_na=False).mean())
        etf_list[i]['RSI'] = RSI(etf_list[i]['close'], 14)
        etf_list[i] = etf_list[i][15:-1]
    return etf_list

In [6]:
def save_metrics(y_true, y_pred, ticker):
    accuracy = accuracy_score(y_true, y_pred)
#     precision = precision_score(y_true, y_pred)
#     f1 = f1_score(y_true, y_pred)
    roc = roc_auc_score(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    conf = confusion_matrix(y_true, y_pred)
    return {'Ticker':ticker,'Accuracy':accuracy,'ROC AUC':roc,'R2':r2,'mse':mse,'conf':conf}

def plot_confusion_matrix(cm, classes, normalize=False, title='Confusion matrix', cmap=plt.cm.Blues):
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()

def produce_confs(results):
    plt.figure()    
    for i in results:
        plot_confusion_matrix(i['conf'], classes =['down', 'up'], title=i['Ticker']+' Predicted Movement')
        plt.show()

In [9]:
def time_test_split(X, y, date):
    scaler = StandardScaler()
    X_train = X[X['date'] < date].drop(columns='date')
    scaler.fit(X_train)
    X_test = X[X['date'] >= date].drop(columns='date')
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)
    y_train = np.array(y[y['date'] < date].drop(columns='date')).ravel().astype('int')
    y_test = np.array(y[y['date'] >= date].drop(columns='date')).ravel().astype('int')
    return X_train, X_test, y_train, y_test

def plant_forests(df_list, split_date):
    results = []
    importances = []
    for i in range(len(df_list)):
        ticker = df_list[i]['ticker'].iloc[0]
        X = df_list[i].reset_index().drop(columns=['ticker','open', 'Clf_Target'])
        feat_labels = X.drop(columns='date').columns
        y = df_list[i]['Clf_Target'].reset_index()
        X_train, X_test, y_train, y_test = time_test_split(X, y, split_date)
        clf = RandomForestClassifier(n_estimators=100)
        clf.fit(X_train, y_train)
        y_pred = clf.predict(X_test)
        results.append(save_metrics(y_test, y_pred, ticker))
        importances.append((ticker, {i:j for i, j in zip(feat_labels, clf.feature_importances_)}))
    return results, importances

def construct_SVMS(df_list, split_date):
    results = []
    for i in range(len(df_list)):
        ticker = df_list[i]['ticker'].iloc[0]
        X = df_list[i].reset_index().drop(columns=['ticker','open', 'Clf_Target'])
        y = df_list[i]['Clf_Target'].reset_index()
        X_train, X_test, y_train, y_test = time_test_split(X, y, split_date)
        clf = SVC()
        clf.fit(X_train, y_train)
        y_pred = clf.predict(X_test)
        results.append(save_metrics(y_test, y_pred, ticker))
    return results

def logistic_regression(df_list, split_date):
    results = []
    for i in range(len(df_list)):
        ticker = df_list[i]['ticker'].iloc[0]
        X = df_list[i].reset_index().drop(columns=['ticker','open', 'Clf_Target'])
        y = df_list[i]['Clf_Target'].reset_index()
        X_train, X_test, y_train, y_test = time_test_split(X, y, split_date)
        clf = LogisticRegression()
        clf.fit(X_train, y_train)
        y_pred = clf.predict(X_test)
        results.append(save_metrics(y_test, y_pred, ticker))
    return results

def XG_Boost(df_list, split_date):
    results = []
    parameters = []
    for i in range(len(df_list)):
        ticker = df_list[i]['ticker'].iloc[0]
        X = df_list[i].reset_index().drop(columns=['ticker','open', 'Clf_Target'])
        y = df_list[i]['Clf_Target'].reset_index()
        X_train, X_test, y_train, y_test = time_test_split(X, y, split_date)
        clf = XGBClassifier()
#         clf.fit(X_train, y_train)
#         y_pred = clf.predict(X_test)
#         results.append(save_metrics(y_test, y_pred, ticker))
        param_grid = {"learning_rate": [.05, .01, .05],
            'max_depth': [10, 15, 20],
            'min_child_weight': [10],
            'subsample': [.7],
            'n_estimators': [50, 100, 250, 350]}
        grid_clf = GridSearchCV(clf, param_grid)
        grid_clf.fit(X_train, y_train)
        val_preds = grid_clf.predict(X_test)
        results.append(save_metrics(y_test, val_preds, ticker))
        best_parameters = grid_clf.best_params_
        parameters.append({ticker:best_parameters})
    return results, parameters

In [10]:
split = '2018-09'
def run_models(window):
    data = clean_data(get_iex_data(etf_list))
    data = add_lags(data, window)
    data = add_indicators(data)
    rf_results, rf_importances = plant_forests(data, split)
    svm_results = construct_SVMS(data, split)
#     lr_results = logistic_regression(data, split)
    xg_results, parameters = XG_Boost(data,split)
    return rf_results, rf_importances, svm_results, xg_results, parameters

In [11]:
rf_results, rf_importances, svm, xg, parameters = run_models(5)

/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning:

The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.

/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning:

The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.

/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning:

The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.

/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning:

The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use

/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning:

The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.

/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning:

The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.

/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning:

The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.

/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning:

The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use

/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning:

The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.

/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning:

The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.

/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning:

The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.

/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning:

The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use

/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning:

The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.

/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning:

The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.

/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning:

The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.

/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning:

The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use

/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning:

The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.

/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning:

The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.

/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning:

The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.

/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning:

The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use

...

/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning:

The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.

/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning:

The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.

/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning:

The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.

/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning:

The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use

/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning:

The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.

/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning:

The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.

/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning:

The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.

/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning:

The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use

/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning:

The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.

/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning:

The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.

/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning:

The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.

/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning:

The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use

/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning:

The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.

/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning:

The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.

/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning:

The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.

/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning:

The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use

/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning:

The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.

/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning:

The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.

/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning:

The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.

/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning:

The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use

In [14]:
rf_results_df = round(pd.DataFrame(rf_results).drop(columns = ['conf']),4)
svm_results_df = round(pd.DataFrame(svm).drop(columns = ['conf']),4)
xg_results_df = round(pd.DataFrame(xg).drop(columns = ['conf']),4)

In [15]:
rf_confs = {i['Ticker']: i['conf'] for i in rf_results}
xg_confs = {i['Ticker']: i['conf'] for i in xg}
svm_confs = {i['Ticker']: i['conf'] for i in svm}

## Predicting direction with RF, SVM, XGBoost

In [16]:
def classification_metrics(ETFs):
    trace = go.Table(header=dict(values=['','Random Forest', 'SVM', 'XGBoost']),
        cells=dict(values=[['Accuracy', 'R-Squared', 'ROC-AUC'], [rf_results_df[rf_results_df['Ticker']==ETFs]['Accuracy'], rf_results_df[rf_results_df['Ticker']==ETFs]['R2'], [rf_results_df[rf_results_df['Ticker']==ETFs]['ROC AUC']]],
                          [svm_results_df[svm_results_df['Ticker']==ETFs]['Accuracy'], svm_results_df[svm_results_df['Ticker']==ETFs]['R2'], [svm_results_df[svm_results_df['Ticker']==ETFs]['ROC AUC']]],
                          [xg_results_df[xg_results_df['Ticker']==ETFs]['Accuracy'], xg_results_df[xg_results_df['Ticker']==ETFs]['R2'], [xg_results_df[xg_results_df['Ticker']==ETFs]['ROC AUC']]]
                          ]))
    layout = {'height':270}
#     trace2 = go.Table(header=dict(values=['','Random Forest Conf', 'SVM Conf', 'XGBoost Conf']),
#         cells=dict(values=[forest, vector, boost]))
    data = [trace] 
    fig = go.Figure(data=data, layout=layout)
    iplot(fig)
    plot_confusion_matrix(rf_confs[ETFs], classes=['down', 'up'], title = 'Random Forest '+ETFs+' Price Classification')
    plt.show()
    plot_confusion_matrix(svm_confs[ETFs], classes=['down', 'up'], title = 'SVM '+ETFs+' Price Classification')
    plt.show()
    plot_confusion_matrix(xg_confs[ETFs], classes=['down', 'up'], title = 'XGBoost '+ETFs+' Price Classification')
    plt.show()
interact(classification_metrics, ETFs=etf_list)

interactive(children=(Dropdown(description='ETFs', options=('SPY', 'IVV', 'VTI', 'VOO', 'QQQ', 'VEA', 'EFA', 'IEFA', 'VWO', 'AGG', 'IJH', 'IEMG', 'IWM', 'IJR', 'VTV', 'IWF', 'IWD', 'VUG', 'BND', 'LQD'), value='SPY'), Output()), _dom_classes=('widget-interact',))

<function __main__.classification_metrics>

In [25]:
data[0]

,open,high,low,close,volume,ticker,Clf_Target,1day_change,2day_change,3day_change,4day_change,5day_change,ewma7,ewma50,ewma200,MACD,RSI
date,,,,,,,,,,,,,,,,,
2013-10-25,159.0222,159.4661,158.7141,159.4208,93571048,SPY,1.0,-0.7248,-1.2503,-0.4892,-1.4043,-1.4152,158.070794,155.829493,155.441673,0.952123,69.670851
2013-10-28,159.3665,159.8920,159.1943,159.6745,84672528,SPY,1.0,-0.2537,-0.9785,-1.5040,-0.7429,-1.6580,158.474757,156.135078,155.711077,1.006114,70.205362
2013-10-29,160.0370,160.5897,159.8105,160.5262,87248072,SPY,0.0,-0.8517,-1.1054,-1.8302,-2.3557,-1.5946,158.990525,156.470560,156.001925,1.088811,71.990038
2013-10-30,160.7165,160.8343,159.1581,159.7289,139662588,SPY,0.0,0.7973,-0.0544,-0.3081,-1.0329,-1.5584,159.175903,156.710572,156.216233,1.087976,67.890365
2013-10-31,159.6021,160.2725,159.0403,159.2759,133367094,SPY,1.0,0.4530,1.2503,0.3986,0.1449,-0.5799,159.200982,156.893245,156.384183,1.044674,65.604393
2013-11-01,159.4843,160.0188,158.7594,159.6564,142775416,SPY,1.0,-0.3805,0.0725,0.8698,0.0181,-0.2356,159.315108,157.083904,156.556076,1.023871,66.621055
2013-11-04,160.0913,160.2816,159.4503,160.2182,85486867,SPY,0.0,-0.5618,-0.9423,-0.4893,0.3080,-0.5437,159.541284,157.293918,156.740593,1.033249,68.119400
2013-11-05,159.5930,160.1457,159.0765,159.7108,85601525,SPY,1.0,0.5074,-0.0544,-0.4349,0.0181,0.8154,159.583720,157.451482,156.884432,0.993800,65.269668
2013-11-06,160.3994,160.8252,159.9554,160.5262,87129195,SPY,0.0,-0.8154,-0.3080,-0.8698,-1.2503,-0.7973,159.819577,157.646857,157.054259,1.007488,67.614384
